# Game sessions

## Preparation

In [ ]:
%run "../Utilities/Preparation.ipynb"
%run "../Static data/Google form correct answers.ipynb"

## Functions

In [ ]:
def getAllSessions( df, dropna ):
    _result = df.loc[:, renamedRelevantColumns]
    _result = _result[_result['type']=='start']
    _result = _result.drop('type', 1)
    if dropna:
        _result = _result.dropna(how='any')
    return _result

def getAllSessionsOfUser( df, userId, dropna ):
    _sessionsList = getAllSessions( df, dropna )
    return getAllSessionsOfUserOptimized(_sessionsList, userId)

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getAllSessionsOfUserOptimized (sessionsList, userId):
    return sessionsList[sessionsList['userId']==userId]

# includewithoutusers=True will count sessions that do not have any userId attached
def countSessions( df, userId, includewithoutusers):
    return getAllSessionsOfUser( df, userId, includewithoutusers )['sessionId'].nunique()
    #return getAllSessionsOfUser( df, userId, includewithoutusers ).shape[0]

# sessionsList is assumed to contain the dataframe of userIds and sessionsIds
def getSessionsCountOptimized( df, userId):
    return getAllSessionsOfUserOptimized( df, userId ).shape[0]

def getUserSessionsCounts( df ):
    _allSessions = getAllSessions( df, True )
    return _allSessions.groupby('userId').size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [ ]:
# get datetime of first significant event
# _userId is assumed to be in RedMetrics format
def getFirstEventDate( _userId ):
    _sessions = getAllSessionsOfUser(df152, _userId, True)
    
    _firstGameTime = pd.Timestamp.max.tz_localize('utc')

    for session in _sessions['sessionId']:
        _timedEvents = df152[df152['sessionId']==session]
        _timedEvents = _timedEvents.dropna(subset=['section'])
        
        if(len(_timedEvents) > 0):
            _timedEvents['userTime'] = _timedEvents['userTime'].map(lambda t: pd.to_datetime(t, utc=True))

            _earliest = _timedEvents['userTime'].min()
            if(_earliest < _firstGameTime):
                _firstGameTime = _earliest

    return _firstGameTime